In [1]:
import sys
sys.path.append('../')

In [3]:
from my_notion_companion.chatbot import ChatBot
from my_notion_companion.self_query import SelfQueryAgent

In [4]:
from langchain_community.llms import LlamaCpp

import tomllib

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+'Qwen-7B-Chat.Q4_K_M.gguf',
    name='Qwen/Qwen-7B-Chat', 
    **_CONFIGS['llm']
)

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 259 tensors from /Users/fred/Documents/models/Qwen-7B-Chat.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen
llama_model_loader: - kv   1:                               general.name str              = Qwen
llama_model_loader: - kv   2:                        qwen.context_length u32              = 32768
llama_model_loader: - kv   3:                           qwen.block_count u32              = 32
llama_model_loader: - kv   4:

In [5]:
c = ChatBot(llm, _CONFIGS)
s = SelfQueryAgent(llm, _CONFIGS, _TOKENS)

In [6]:
c.invoke("who are you?")


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format (without BOS/EOS tokens!). If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


llama_print_timings:        load time =    7970.19 ms
llama_print_timings:      sample time =      26.93 ms /    41 runs   (    0.66 ms per token,  1522.75 tokens per second)
llama_print_timings: prompt eval time =    7969.76 ms /   127 tokens (   62.75 ms per token,    15.94 tokens per second)
llama_print_timings:        eval time =    3302.14 ms /    40 runs   (   82.55 ms per token,    12.11 tokens per second)
llama_print_timings:       total time =   11610.52 ms /   167 tokens


{'role': 'assistant',
 'content': 'I am QianWen, a large language model created by Alibaba Cloud. I am here to assist you with any questions or tasks you may have. How can I help you today?[PAD151645]\n'}

In [10]:
import re

with open('../data/test_cases.txt') as f:
    test_cases_raw = f.readlines()

test_cases_raw = "".join(test_cases_raw[0:-1:2]).split('问：')[1:]
test_cases_raw = [re.split(r'\n答：|\n资料：', x) for x in test_cases_raw]
test_cases = [{'q': x[0], 'a': x[1], 'docs': x[2:]} for x in test_cases_raw]

retrival_sample_questions = [x['q'] for x in test_cases]
retrival_sample_questions[:3]

['什么是我国第一部编年史著作？', '什么是我国第一部编年国别史？', '“寡人之于国也”下一句是什么？来自哪里？']

In [13]:
%%time

retrival_sample_embeddings = s.embedding_model.embed_documents(retrival_sample_questions, )

CPU times: user 31.9 ms, sys: 5.07 ms, total: 37 ms
Wall time: 177 ms


In [21]:
from langchain.utils.math import cosine_similarity

query_embedding = s.embedding_model.embed_query("who are you?")
cosine_similarity([query_embedding], retrival_sample_embeddings)[0].max()

0.27933269930385096

In [ ]:
https://python.langchain.com/docs/expression_language/how_to/routing

In [22]:
from langchain_core.runnables import RunnableLambda

THRESHOLD = 0.5

def route(input):

    query_embedding = s.embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], retrival_sample_embeddings)[0]

    if similarity >= THRESHOLD:
        chain = s
    else:
        chain = c

    return chain



full_chain = {"topic": chain, "question": lambda x: x["query"]} | RunnableLambda(
    route
)

NameError: name 'chain' is not defined